In [7]:
# main.py


from simulation import build_hamiltonian, cut_circuit, run_exact_sampler, run_mps_simulator, reconstruct_expectation, trotterization_circuit
from utils import circuit_data_dict, save_to_json

from qiskit.transpiler import CouplingMap
# from qiskit.circuit import Parameter
# from qiskit import transpile
from qiskit.quantum_info import SparsePauliOp
import numpy as np
import matplotlib.pyplot as plt


In [38]:
def main(config):

    # Define Copling map for qubits and total number of qubits
    num_qubits = config.NUM_RINGS * config.NUM_SPINS
    edges = config.EDGES
    coupling_map = CouplingMap(edges)

    # Build Hamiltonian
    H = build_hamiltonian(
        num_qubits=num_qubits,
        coupling_map=coupling_map,
        decrease=config.DECREASING,
        single_gates=config.SINGLE_GATES_LIST,
        two_gates=config.TWO_GATES_LIST,
        anisotropy=config.ANISOTROPY,
        h=config.H_FIELD
    )

    #Obtain evolution circuit under TrotterLie approximation
    evolutionQC = trotterization_circuit(hamiltonian = H, trotter_reps = config.TROTTER_REPS, dt = config.DT)
    print('Obtained full circuit')
    # Define Observables (for now Z on each site, ZII + IZI + IIZ)
    z_list = [('I', [i], 1.0) for i in range(num_qubits)]
    z_observables = SparsePauliOp.from_sparse_list(z_list, num_qubits=num_qubits)

    # Cut circuit and generate subexperiments
    cutting_labels = config.NUM_SPINS * 'A' + config.NUM_SPINS * 'B' #Use this for ring model
    # cutting_labels = int(config.NUM_SPINS/2) * 'A' + int(config.NUM_SPINS/2) * 'B' #Use this for line model
    # cutting_labels = 'A' + (config.NUM_SPINS-1)*'B' # Use this for line model
    subexperiments, coefficients, subobservables = cut_circuit(
        circuit=evolutionQC,
        partition_labels=cutting_labels,
        observables=z_observables,
        num_samples=config.N_SAMPLES
    )
    print('Obtained cut circuits')
    k = 0
    print(coefficients)
    for i, coeff in enumerate(coefficients):
        k += coeff[0]
    
    for i,coeff in enumerate(coefficients):
        coefficients[i] = (coeff[0]/k,coeff[1])
    print(coefficients)
    # Run Simulations
    results = run_exact_sampler(subexperiments)

    # print(results)
    # # print(results['A'])
    # print('Run simulations')
    # results2 = run_mps_simulator(subexperiments)
    # print(results2)
    # print('Run tensor simulation')
    # # Reconstruct Expectation Values
    reconstructed_expval = reconstruct_expectation(
        results, coefficients, subobservables, z_observables
    )
    print(reconstructed_expval)
    # reconstructed_expval = 0
    print('Reconstructed expval')
    # Generate experiment and subcircuit data dictionary, and save to JSON
    basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
    data_dict = circuit_data_dict(subexperiments, coefficients, reconstructed_expval, basis_gates)
    save_to_json(data_dict)

class Config:
    #Class containing the configuration parameters
    def __init__(self):
        # Hamiltonian parameters
        self.NUM_SPINS = 3  # Number of qubits per ring #I am not able to go past 15! 
        # I have checked and the problematic step is simulation. Let's try MPS Simulator!
        self.NUM_RINGS = 2  # Total number of rings 
        self.ANISOTROPY = 1.02 #Variable J
        self.H_FIELD = 1.0
       
        # Gates in the Hamiltonian
        self.TWO_GATES_LIST = ['XX']
        self.SINGLE_GATES_LIST = ['Z']
        
        # Hamiltonian coupling map edges

        ### MODEL: DOUBLE RING
        self.EDGES = [
            (ring * self.NUM_SPINS + i, ring * self.NUM_SPINS + (i + 1) % self.NUM_SPINS)
            for ring in range(self.NUM_RINGS)
            for i in range(self.NUM_SPINS)
        ] + [(0, 2 * self.NUM_SPINS - 1)]  # The coupling map is unidirectional. Does this affect dynamics?

        ### MODEL: LINE
        # self.EDGES = [(i, (i+1) % self.NUM_SPINS) for i in range(self.NUM_SPINS-1)]
        #Evolution parameters
        self.TROTTER_REPS = 1
        self.DT = 0.1  # Time parameter

        self.DECREASING = False # Flag for decreasing the intensity of J with distance to the first qubit

        # Sampling

        # self.N_SAMPLES = np.inf
        self.N_SAMPLES = 4



if __name__ == "__main__":
    
    config = Config()

    main(config)


Obtained full circuit
Obtained cut circuits
[(0.9896320310575152, <WeightType.EXACT: 1>), (-0.20777198177110615, <WeightType.SAMPLED: 2>), (-0.20777198177110615, <WeightType.SAMPLED: 2>)]
[(1.723833131283704, <WeightType.EXACT: 1>), (-0.3619165656418521, <WeightType.SAMPLED: 2>), (-0.3619165656418521, <WeightType.SAMPLED: 2>)]
(10.34299878770222+0j)
Reconstructed expval
Subcircuits data has been successfully saved to 'subcircuits.json'.
